In [ ]:
from deepface import DeepFace
import numpy as np
import os
import pickle
import time
import json
import cv2
import matplotlib.pyplot as plt

ENCODING_FILE = "encodings_deepface/deepface_encodings.pkl"
KNOWN_DIR = "known_faces/dataset_imgs"


def build_encodings():
    encodings = []
    names = []

    files = [f for f in os.listdir(KNOWN_DIR) if f.lower().endswith((".jpg",".png",".jpeg"))]

    for file in files:
        path = os.path.join(KNOWN_DIR, file)
        try:
            rep = DeepFace.represent(
                img_path=path,
                model_name="Facenet512",
                enforce_detection=True
            )

            encodings.append(rep[0]["embedding"])
            names.append(os.path.splitext(file)[0])

        except:
            print("Skipped:", file)

    with open(ENCODING_FILE, "wb") as f:
        pickle.dump((encodings, names), f)

    print("✅ DeepFace encodings created")


def load_known_faces():
    if not os.path.exists(ENCODING_FILE):
        build_encodings()

    with open(ENCODING_FILE, "rb") as f:
        return pickle.load(f)

def recognize_face(image_path):
    start = time.time()

    known_faces, known_names = load_known_faces()

    try:
        reps = DeepFace.represent(
            img_path=image_path,
            model_name="Facenet512",
            enforce_detection=True
        )
    except:
        return {"status": "no_face_detected"}

    input_embedding = np.array(reps[0]["embedding"])

    distances = np.linalg.norm(np.array(known_faces) - input_embedding, axis=1)
    best_match = distances.argmin()
    best_distance = distances[best_match]

    # Display image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.axis("off")
    plt.title("Face Processed")
    plt.show()

    if best_distance < 0.70:
        return {
            "status": "matched",
            "person": known_names[best_match],
            "confidence": round(1 - best_distance, 3),
            "time_taken_sec": round(time.time() - start, 3)
        }

    return {
        "status": "not_matched",
        "time_taken_sec": round(time.time() - start, 3)
    }

# Run test
result = recognize_face("mudassir.jpeg")
print(json.dumps(result, indent=2))


{
  "status": "no_face_detected"
}
